In [ ]:
import os
import pandas as pd
import requests # library for getting webpages (and a ton of other stuff, definitely something look into)
from lxml import html # lxml is a XML parsing library, they have a submodule (look it up) called html that parses HTML.

# Getting and Extracting Data

In [ ]:
sidenote = """
HTML is a semi-structured data format. The data is hidden between tags and particularly useful is that the data 
we're grabbing exists in a HTML table so it's even more structured. But we must nevertheless extract it. Feel free to steal
the below function for your own scraping. Or make it better. Or do whatever you want, I don't care.

Scenario 1 (note the presence of tbody):
<table>
    <tbody>
        <tr>
            <td> item1 </td>
            <td> item2 </td>
            <td> item3 </td>
        </tr>
    </tbody>
</table>

xpath to get the table --> //table/tbody
================================
Scenario 2:

<table>
    <tr>
        <td> item1 </td>
        <td> item2 </td>
        <td> item3 </td>
    </tr>
</table>

xpath to get the table --> //table

"""

In [ ]:
def extract_table(htm_tbl):
    rows = []
    
    # make sure the xpath object stored in the htm_tbl parameter and is passed into the function is at the table level
    # if there is a tbody element then include that in the xpath so that you can just access the <tr> elements directly. 
    # depending on the author of the website, they might use the tbody or they might use just table and go directly into <tr>
    
    trs = htm_tbl.xpath('tr') # this is calling out all of the tr elements
    for row in trs:
        nR = [] # this is an empty list to be used below
        tds = row.xpath('td')
        for td in tds:
            nR.append(td.text_content()) # this is doing two things! Can you guess what it is? (think order of operations even)
            # it's taking the td variable, which is storing the td element, and is accessing the text attribute
            # it is then appending that to the nR list we created
        if len(nR) > 0:
            rows.append(nR) # this is placing the nR list into the rows list making a list of lists!
    return rows

In [ ]:
def getData():
    # Here we're getting the data we need for below, but won't actually discuss web scraping until later...

    url = r'https://en.wikipedia.org/wiki/List_of_members_of_the_Forbes_400'
    r = requests.get(url) # this is using the get function in the requests library that gets the HTML of a website
    print(r)
    body = html.fromstring(r.content) # this is putting the content into a parsable format

    xp = r'//*[@id="mw-content-text"]/table[2]' # this is an xpath. Look it up.
    tbl = body.xpath(xp)
    
    
    data = extract_table(tbl[0])
    
    
    # okay so it's not perfect, we still have to process it (the first element is wonky and the net worth isn't there)
    # also you can handle the header in two ways you can tell python to grab it by accessing the <th> element or you can
    # just hand write it like below:
    header = ['rank', 'Name', 'Net worth (USD)', 'Sources of wealth']
    # You'll notice the first element brought in a bit of noise, that's no fun
    # what do you notice? Can you tell if there is a way to extract it?
    # the easy thing to do since it's just a sequence is to replace it with the sequence and do away with that noise, 
    # but we'll entertain the idea that it's not that easy...

    example = ['7000100000000000000♠1', 'Bill Gates', '$76\xa0billion ', 'Microsoft, Cascade Investments LLC']
    ['7000200000000000000♠2', 'Warren Buffett', '$62\xa0billion ', 'Berkshire Hathaway']
    ['7000300000000000000♠3', 'Larry Ellison', '$47.5\xa0billion ', 'Oracle Corporation']

    
    rows = [] # this is an empty container to place the rows
    
    for row in data:
        
        if len(row) > 0:
            row[0] = row[0].split('♠')[1] # this is cleaning the first element in the list

            row[2] = row[2].replace('\xa0billion ', '').replace('$', '')
            row[2] = float(row[2]) * 1000000000    

            rows.append(row)
    return header, rows

In [ ]:
header, billionaires = getData()
for billionaire in billionaires:
    print(billionaire)

## We're going to explore all of these through a nifty searcher

We'll be doing this through various functions (that we write) and manipulate the data. (And we're going to do it mostly live...)

We'll be going on a journey from a single list of names with some attributes to a list of 

### First things first

First, we have to go to Google and set up an API and what not and I'll show you how to do that. Keep in mind that this isn't necessarily free, but it's pretty cheap; allowing you to do 20+ searches per name! 

You can use this for farming which folks to focus on. Those with the most number of results on strict search parameters will be those you might want to focus on. 

### Down and dirty

See below for a skeleton of functions...

Now we're getting into programming! You previously learned just the syntax. The art of programming and the more difficult part is aligning your thinking appropriately. As researchers, we're pretty good at problem solving. We have to create something out of seemingly nothing. We take a breadcrumb and extrapolate the meaning from it with regard to a prospect. We are constantly moving from big picture to the granular and back. This is what programming is on a much more extreme scale. Programming is problem solving.

In programming you have to deconstruct a problem into its smallest form, then solve that and then solve the next level and so on until the bigger question or problem is answered. 

Our problem, we have 200 names, and we want to search google for each of them, using all of the variations we can think of. 

What do we need to do first? What are we missing? 

In [ ]:
def extract_terms(row):
    
    id_ = row[0]
    name = row[1]
    name = '"{}"'.format(name) # this puts quotes around the search term
    src = row[-1] # you can access the last element of a list by using -1
    
    search_terms = ["{name} {src}".format(name=name, src=src)]
    
    other_terms = [
        'John Connelly', 'Rochester Institute of Technology', 'Art Institute of Chicago'
        ] # this is a list for any other terms you'd like to add
    
    sites = [
        'site:bloomberg.com', 'site:forbes.com'
        ]
    
    for ot in other_terms:
        item = '"{}"'.format(ot)
        search_term = "{name} {item}".format(item=item, name=name)
        search_terms.append(search_term)
        
    for s in sites:
        item = '"{}"'.format(s)
        search_term = "{name} {item}".format(item=item, name=name)
        search_terms.append(search_term)
    
    return search_terms

In [ ]:
extract_terms(billionaires[0])

In [ ]:
def basic_google_search(query, start=1):
    api_key = '' # Get your own API key from google after enabling the custom serach API 
    cx = '001917876160815680181:4fyirrxtkq4'
    u = r'https://www.googleapis.com/customsearch/v1'
        
    params = {
            'cx': cx,
            'q': query,
            'key': api_key,
            'start': start
        }
    r = requests.get(u, params=params)
    return r.json()

In [ ]:
def google_search(query, pages=5):
    results = []
    formatted_results = []
    urls = []
    cnt = 1
    dbl = 0
    for p in range(pages):

        if p == 0:
            p = 1
        else:
            p *= 10

        google_results = basic_google_search(query, start=p)
        items = google_results.get('items')
        if items is not None:
            for ix, res in enumerate(items):
                res['query'] = query
                results.append(res)
                #print(res.keys())
                link = res.get('link')
                htmlFormattedUrl = res.get('htmlFormattedUrl')
                htmlSnippet = res.get('htmlSnippet')
                formattedUrl = res.get('formattedUrl')
                htmlTitle = res.get('htmlTitle')
                kind = res.get('kind')
                displayLink = res.get('displayLink')
                title = res.get('title')
                cacheId = res.get('cacheId')
                snippet = res.get('snippet')
                pagemap = res.get('pagemap')

                if link not in urls:
                    urls.append(link)
                    formatted_results.append([title, link, snippet, query])
                    cnt += 1
                else:
                    dbl += 1
                                    
    return results, formatted_results

In [ ]:
header, billionaires = getData()

search_terms = []
for billionaire in billionaires:
    print(billionaire[1])
    terms = extract_terms(billionaire)
    for term in terms:
        print('\t', term)
    billionaire.append([]) # this places an empty list in the billionaire row to store our search terms
    for row in terms:
        billionaire[-1].append(row)
    
    search_terms.append(billionaire)
    

In [ ]:
google_results = []
for entity in search_terms:
    print(entity[:-1])
    terms = entity[-1]
    for term in terms:
        print('\t', term)
        results, formatted_results = google_search(term, pages=2)
        for res in formatted_results:
            nR = entity[:-1] + res
            google_results.append(nR)
            print('\t\t', res[0])

In [ ]:
for gr in google_results:
    print(gr)

In [ ]:
header = ['rank', 'name', 'net worth', 'source of wealth', 'search_title', 'search_url', 'search_blurb', 'search_term']
df = pd.DataFrame(google_results, columns=header)

In [ ]:
writer = pd.ExcelWriter('google_results_billionaires.xlsx')
df.to_excel(writer)
writer.save()